In [1]:
from rembg.bg import remove
import numpy as np
import io
from PIL import Image
import cv2
from tqdm import tqdm

In [2]:
t_direct = "dataset/test-data/"

In [6]:
print("Removing Background....")
for i in tqdm(range(1,21)):
    path = t_direct+str(i)+".png"
    f = np.fromfile(path)
    result = remove(f)
    img = Image.open(io.BytesIO(result)).convert("RGBA")
    img.save(path)
    
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # do adaptive threshold on gray image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 101, 3)

    # apply morphology open then close
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
    blob = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    blob = cv2.morphologyEx(blob, cv2.MORPH_CLOSE, kernel)

    # invert blob
    blob = (255 - blob)

    # Get contours
    cnts, hierarchy = cv2.findContours(blob, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    result1 = img.copy()
    cv2.drawContours(result1, cnts, -1, (0, 0, 255), 1)
    path = t_direct+str(i)+"-a.png"
    cv2.imwrite(path, result1)

print("Background removed....")

Removing Background....


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.46s/it]

Background removed....
